# Analisis de Audios con Gemini
En este Notebook, analizamos con ayuda de Gemini y Python los audios de las agencias de cobranza externa de la cartera castigada y vencida.
Necesitaremos acceso a nuestro Drive y a una "Key" de Gemini, con la cual se realizará el analísis.
<br>
<br>
Primero realizaremos unas pruebas para conectar a Drive y comprobar que tenemos permisos de edición sobre la carpeta:

In [ ]:
# 1. CONEXIÓN DE GOOGLE DRIVE
from google.colab import drive
print("Intentando montar Google Drive...")
try:
    drive.mount('/content/drive', force_remount=True)
    print("✅ Google Drive montado exitosamente.")
except Exception as e:
    print(f"❌ Error al montar Drive: {e}")


In [ ]:
import os
folder_to_check = '/content/drive/Shared drives/mi_ruta/Empresa1/'
print(f"Verificando carpeta: {folder_to_check}")
if os.path.exists(folder_to_check):
    print("✅ Existe")
else:
    print("❌ No existe")

### Configuración de Gemini y Proceso de Auditoría

In [ ]:
import os
import glob
import unicodedata
import time
from google import genai
from google.colab import drive

API_KEY = "Mi_APY_KEY"
MODELO_PROCESO = "gemini-2.0-flash"
RUTA_BASE = "/content/drive/Shared drives/mi_ruta/Grabaciones/Empresa1"
RUTA_LINEAMIENTOS = "/content/drive/Shared drives/mi_ruta/Lineamientos.pdf"
ARCHIVO_SALIDA = os.path.join(RUTA_BASE, "RESULTADOS_AUDITORIA.txt")

client = genai.Client(api_key=API_KEY)

def limpiar_nombre_archivo(nombre):
    nfkd = unicodedata.normalize("NFKD", nombre)
    return "".join([c for c in nfkd if not unicodedata.combining(c)]).replace(" ", "_")

def subir_con_diagnostico(ruta_archivo):
    try:
        f = client.files.upload(file=ruta_archivo, config={"display_name": limpiar_nombre_archivo(os.path.basename(ruta_archivo))})
        while f.state.name == "PROCESSING":
            time.sleep(5)
            f = client.files.get(name=f.name)
        return f
    except Exception as e:
        print(f"Error: {e}")
        return None

def ejecutar_auditoria():
    print("=== INICIANDO ===")
    file_pdf = subir_con_diagnostico(RUTA_LINEAMIENTOS)
    if not file_pdf: return
    print("PDF Listo.")

ejecutar_auditoria()

In [ ]:
import os
import glob
carpeta = "/content/drive/Shared drives/miruta/Empresa1"
os.chdir(carpeta)
extensiones = ["*.mp4", "*.mp3", "*.aac", "*.m4a"]
for ext in extensiones:
  for archivo in glob.glob(ext):
    base = os.path.splitext(archivo)[0]
    salida = base + ".wav"
    os.system(f'ffmpeg -y -i "{archivo}" -ar 16000 -ac 1 "{salida}"')
    os.remove(archivo)
    print(f"Convertido: {archivo}")